In [1]:
import paddle
import paddle.fluid as fluid
import paddle.dataset as pd
import numpy as np

from paddle.fluid.dygraph.nn import Conv2D, Pool2D
from paddle.fluid.optimizer import AdamOptimizer
from paddle.fluid.dygraph.base import to_variable

In [2]:
# Defining hyperparameter

BATCH_SIZE = 64
RL = 0.001
EPOCHS = 10
FULL_SIZE = 10
USE_CUDA = True
ACT = "relu"


In [3]:
# Constructing the reader
train_reader = paddle.batch(
    pd.mnist.train(), batch_size=BATCH_SIZE, drop_last=True)
test_reader = paddle.batch(
    pd.mnist.test(), batch_size=BATCH_SIZE, drop_last=True)

In [4]:
class MNIST(fluid.dygraph.Layer):
    def __init__(self):
        super(MNIST, self).__init__()

        # class paddle.fluid.dygraph.Conv2D(num_channels, num_filters, filter_size, stride=1, padding=0, dilation=1, groups=None, param_attr=None, bias_attr=None, use_cudnn=True, act=None, dtype='float32')
        self.conv2d_1 = Conv2D(1,20,5,use_cudnn=USE_CUDA,act=ACT,)
        # class paddle.fluid.dygraph.Pool2D(pool_size=-1, pool_type='max', pool_stride=1, pool_padding=0, global_pooling=False, use_cudnn=True, ceil_mode=False, exclusive=True)
        self.pool_1 = Pool2D(2, pool_stride=2,)
        
        self.conv2d_2 = Conv2D(20,50,5,use_cudnn=USE_CUDA,act=ACT,)

        self.pool_2 = Pool2D(2, pool_stride=2,)

        self.pool_2_shape = 50*4*4

        # Defining the parameters of the full connection layer
        self.output_weight = self.create_parameter([self.pool_2_shape, 10])

    def forward(self, inputs, label=None):
        x = self.conv2d_1(inputs)
        x = self.pool_2(x)
        x = self.conv2d_2(x)
        x = self.pool_2(x)

        # forward calc
        x = fluid.layers.reshape(x, shape=[-1, self.pool_2_shape])
        x = fluid.layers.matmul(x, self.output_weight)
        x = fluid.layers.softmax(x)
        if label is not None:
            acc = fluid.layers.accuracy(input=x, label=label)
            return x, acc
        else:
            return x





In [11]:


with fluid.dygraph.guard():
    mnist = MNIST()
    # defining Optimizer as Adam
    adam = AdamOptimizer(learning_rate=RL, parameter_list=mnist.parameters())

    # train
    for epoch in range(EPOCHS):
        for batch_id, data in enumerate(train_reader()):

            # each batch contain data and label
            dy_x_data = np.array([x[0].reshape(1, 28, 28) for x in data]).astype('float32')
            y_data = np.array([x[1] for x in data]).astype('int64').reshape(-1, 1)

            # Convert data to Paddle variable
            img = to_variable(dy_x_data)
            label = to_variable(y_data)

            # forward
            cost, acc = mnist(img, label)

            # loss
            loss = fluid.layers.cross_entropy(cost, label)
            avg_loss = fluid.layers.mean(loss)

            # backward
            avg_loss.backward()

            adam.minimize(avg_loss)

            # clear grad
            mnist.clear_gradients()

            if batch_id % 100 == 0:
                print("Loss at epoch {} step {}: {:}".format(
                    epoch, batch_id, avg_loss.numpy()))

    # save model
    model_dict = mnist.state_dict()
    fluid.save_dygraph(model_dict, "save_temp")